In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from helpers.mv_single_binary import majority_voting
import helpers.algorithms_utils as alg_utils
from helpers.truth_finder_single import truth_finder_single
from helpers.truth_finder import expectation_maximization
import helpers.dawid_skene as dawid_skene
import multiprocessing
import time
import matplotlib.pyplot as plt
from algorithms.smart_stop import decision_function_bayes
from algorithms.smart_stop import cost_estimator_bayes
from algorithms.smart_stop import bayes_prob_in
import sys

### compute and add f1 - fbeta metrics

In [40]:
results = [{'path': './results/simulated_dataset/', 'name': 'all_combinations_results_mv'},
{'path': './results/simulated_dataset/', 'name': 'all_combinations_results_break_tie'},
{'path': './results/abraham/', 'name': 'results_cs_es_varied'},
{'path': './results/agenthunt/', 'name': 'simulated_datasets_results'},
{'path': './results/simulated_dataset/', 'name': 'all_combinations_results_adaptive'},
{'path': './results/real_world/', 'name': 'all_mv'},
{'path': './results/real_world/', 'name': 'all_break_tie'},
{'path': './results/real_world/', 'name': 'all_abraham'},
{'path': './results/agenthunt/', 'name': 'real_world_results'},
{'path': './results/real_world/', 'name': 'all_adaptive'}]

for result in results:
    file = open(result['path']+result['name']+'.csv', "r")
    file_lines = file.readlines()

    new_file = open(result['path']+result['name']+'_full_metrics.csv', "w+")

    for key, line in enumerate(file_lines):
        line = line.replace('\n','')
        if key == 0:
            new_line = line + ',f1,f1_std,fbeta,fbeta_std\n'
            headers = line.split(',')
        else:
            old_line = line.split(',')
            old_line[-1].replace('\n','')
            #11 recall
            #13 precision
            beta = int(old_line[headers.index('loss_ratio')])
            recall = float(old_line[headers.index('recall')])
            precision = float(old_line[headers.index('precision')])
            
            f1 = (2 * precision * recall) / (precision + recall)
            f_beta = (beta**2 + 1) * precision * recall / (beta**2 * precision + recall)

            new_line = line + f",{f1},0,{f_beta},0\n"
        #end if

        new_file.write(new_line)

    new_file.close()

In [19]:
import pickle
tempfile = open("experiment.txt", 'wb')
data = "test"
pickler = pickle.Pickler(tempfile)
pickler.dump(data)
tempfile.close()

In [30]:
z = [1*x for x in [0.01, 0.04]]
for x in z:
    print(x)

0.01
0.04


In [300]:
from helpers.mv_single_binary import majority_voting
from algorithms.smart_stop import decision_function_em
from algorithms.smart_stop import decision_function_mv
from algorithms.smart_stop import decision_function_bayes
from algorithms.smart_stop import decision_function_mv_base

%time

def worker_vote(gt, i, acc):
    if np.random.binomial(1, acc):
        vote = gt[i]
    else:
        vote = 1 - gt[i]
        
    return vote

acc = .75
items_num = 1000
th = .8
cr = 1/5
votes_num = 3

gt = alg_utils.generate_gold_data(items_num, .5)
cf = majority_voting
df = decision_function_mv


t_l = []
t_r = []
t_p = []

for _ in range(10):

    workers_accuracy = alg_utils.simulate_workers(1000, 0, True, acc, base_acc = .5)
    votes = {i:{} for i in range(items_num)}
    votes = {x:{val:[alg_utils.get_worker_vote(workers_accuracy, x, gt, votes)[1]] for val in np.arange(0, votes_num, 1)} for x in range(items_num)} 
    must_get_more_votes = True
    
    results = df(items_num, votes, th, cr, cf)
    items_predicted_classified = alg_utils.get_items_predicted_classified(results)
    must_get_more_votes = len(items_predicted_classified) > 0

    while(must_get_more_votes):

        total_votes_aux = {}
        for i, v in items_predicted_classified.items():           
            worker_id, vote = alg_utils.get_worker_vote(workers_accuracy, i, gt, votes)

            votes[i][worker_id] = [vote]

            total_votes_aux[i] = votes[i]
        #end for

        results = df(len(items_predicted_classified), total_votes_aux, th, cr, cf)

        #Stop when there are no more items that can be classified
        items_predicted_classified = alg_utils.get_items_predicted_classified(results)
        #print(len(items_predicted_classified))
        must_get_more_votes = len(items_predicted_classified) > 0

    #print(votes)

    items_classification = alg_utils.classify_items(votes, gt, cf, .5)

    print("----")


    loss, recall, precision =alg_utils.Metrics.compute_metrics(items_classification, gt, 5)
    t_l.append(loss)
    t_r.append(recall)
    t_p.append(precision)
    
print(np.mean(t_l))
print(np.mean(t_r))
print(np.mean(t_p))

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 596 µs
----
----
----


KeyboardInterrupt: 

In [9]:
.75
0.169
0.95
0.9171729912566118
--
.85
0.195
0.9380000000000001
0.9216768105081089
--
.95
0.048
0.982
0.9939983993597439

'''
.8
mv-mv
0.184
0.9592
0.8556714703885876
mv-bayes
0.1468
0.9508
0.9523127645944451
mv-em
0.2144
0.9396
0.8829743958123392
tf-em
0.451
0.8484
0.857014992134125
tf-bayes
0.41440000000000005
0.8611999999999999
0.8663151858194605
'''


'\nmv-bayes\n0.1468\n0.9508\n0.9523127645944451\nmv-em\n0.2144\n0.9396\n0.8829743958123392\ntf-em\n0.451\n0.8484\n0.857014992134125\ntf-bayes\n0.41440000000000005\n0.8611999999999999\n0.8663151858194605\n'

In [122]:
#20,31,45    ///13-25
i = 13
Psi = alg_utils.input_adapter_single(votes[i])
print(votes[i])
print(gt[i])
truth_finder_single(Psi)

{0: [1], 1: [0], 2: [0], 3: [1], 4: [1], 884: [0], 567: [1], 82: [0], 771: [1], 152: [0], 524: [1], 729: [0], 861: [0]}
1


2.2534749635520452e-37

In [24]:
Psi = alg_utils.input_adapter_single(votes[9])
print(Psi)
truth_finder_single(Psi)

[[(0, 0), (1, 1), (2, 1), (3, 1)]]


0.9999999999999993

In [16]:
x = [np.random.binomial(1, .3) for x in range(100)]
np.mean(x)

0.26

In [35]:
v = {
    0:{0: [1], 1: [1], 2: [1], 3:[1]},
    1:{0: [1], 1: [1], 2: [1], 3:[0]},
    2:{0: [1], 1: [0], 2: [0], 3:[0]}
}
#expectation_maximization(4, 2, alg_utils.input_adapter(v))
cf = majority_voting
ct = .9
r = {i:(((1 - ct) <= cf(alg_utils.input_adapter_single(i_votes)) <= ct)) for i, i_votes in v.items()}
print(r)
f = [i for i, d in r.items() if d == True]
for a in f:
    print(a)

{0: False, 1: True, 2: True}
1
2


In [19]:
from helpers.truth_finder import expectation_maximization
from helpers.dawid_skene import dawid_skene
import helpers.algorithms_utils as alg_utils
import numpy as np

v = {
    2:{0: [1], 1: [1], 2: [0]},
    11:{0: [1], 1: [1], 2: [1]},
    3:{0: [0], 1: [0], 2: [0]},
    5:{0: [1], 1: [0], 2: [0]}
}

#ds = dawid_skene(v)
votes = alg_utils.input_adapter(v)

dong = expectation_maximization(3, 4, votes)

#print(ds)
#print(np.mean(dong[0]))
print(dong)

([0.75, 0.9980392156862745, 0.75], [defaultdict(<class 'float'>, {1: 0.996078431372549, 0: 0.0039215686274509725}), defaultdict(<class 'float'>, {1: 1.0}), defaultdict(<class 'float'>, {0: 1.0}), defaultdict(<class 'float'>, {1: 0.0039215686274509725, 0: 0.996078431372549})])


In [224]:
'''
P(positive|evidence) = P(evidence|positive) P(positive) / P(evidence)
= P(evidence|positive) P(positive) / P(evidence|positive) P(positive) + P(evidence|negative) P(negative)
'''
from helpers.truth_finder import expectation_maximization
from scipy.stats import binom
import math

#acc = .66
prior = .5

a = 1
b = 5

v = {
    0:{0: [1], 1: [1], 2: [0]},
    1:{0: [1], 1: [1], 2: [1]},
    2:{0: [0], 1: [0], 2: [0]},
    3:{0: [1], 1: [0], 2: [0]}
}

accs, p = expectation_maximization(3, 4, alg_utils.input_adapter(v))

acc = np.mean(accs)

like_in = binom.pmf(a, a+b, acc)
like_out = binom.pmf(b, a+b, acc)

p_v_pos = math.pow(acc, a) * math.pow(1 - acc, b)
p_v_neg = math.pow(acc, b) * math.pow(1 - acc, a)
#print(like_in)
#print(like_out)
#print(p_v_pos)
#print(p_v_neg)

#print( ((p_v_pos * prior) / ((p_v_pos * prior) + (p_v_neg * (1 - prior)))) )
print( ((like_in * prior) / ((like_in * prior) + (like_out * (1 - prior)))) )


0.0016277011551087206


In [136]:
tst = {
    0:{0: [1], 1: [1], 2: [0]},
    1:{0: [1], 1: [1], 2: [1]},
    2:{0: [0], 1: [0], 2: [0]},
    3:{0: [1], 1: [0], 2: [0]}
}
i = 0
Psi = alg_utils.input_adapter_single(tst[i])
sum([x[1] for x in Psi[0]]) / len(Psi[0])

0.6666666666666666

In [19]:

dts = ['BarzanMozafari', 'RTE', 'SpamCF', 'TEMP', 'WVSCM'] 
for d in dts:
    _, workers_accuracy = alg_utils.get_real_dataset_data(d)
    print(d)
    print(len(workers_accuracy))

BarzanMozafari
83
RTE
164
SpamCF
150
TEMP
76
WVSCM
17


In [509]:
item_votes = {
    0:{1660: [1], 678: [1], 537: [1]},
    1:{1660: [1], 678: [1], 537: [1]},
    2:{1660: [1], 678: [1], 537: [0]},
}

accs, p = expectation_maximization(3, 1, alg_utils.input_adapter(item_votes))
acc_mean = np.mean(accs)
print(acc_mean)

#print(majority_voting(alg_utils.input_adapter_single(item_votes[0])))
print(decision_function_bayes(3, item_votes, .9, 0.05, majority_voting))



1.0
{0: False, 1: False, 2: True}


In [17]:
item_votes = {
    0:{0: [1], 1: [1], 2: [0], 3:[1]},
    1:{0: [1], 1: [1], 2: [1]},
    2:{0: [0], 1: [0], 2: [0]},
    3:{0: [1], 1: [0], 2: [0]}
}
sum([len(v) for i, v in item_votes.items()])

13

In [171]:
import math
from scipy.stats import beta

def gal_uncertainty(a, b, x = .5):
    result = 1
    a = a + 1
    b = b + 1
    for j in np.arange(1, a + b, 1):
        print(j)
        term_1 = math.factorial(a + b - 1) / (math.factorial(j) * math.factorial(a + b - 1 - j))
        term_2 = math.pow(x, j)
        term_3 = math.pow(1 - x, a + b - 1 - j)

        result *= term_1 * term_2 * term_3
        
    return result
    
gal_uncertainty(2, 1)

v_pos = 1 + 3
v_neg = 1 + 1

beta_cdf = beta.cdf(.5, v_pos, v_neg, loc=0, scale=1)

print(beta_cdf)
print(1 - beta_cdf)

min(beta_cdf, 1-beta_cdf)

1
2
3
4
0.1875
0.8125


0.1875

In [102]:
gt = pd.read_csv('data/TEMP/gold.txt', delimiter='\t', header=None)




ground_truth = [str(x) for x in gt[0]]

def get_dataset_next_worker_vote(responses, item_votes, i, categories):
    item_responses = responses[responses[1] == i]
    worker_votes = zip(item_responses[0], item_responses[2])
    
    for worker_id, worker_vote in worker_votes:
        if(worker_id not in item_votes):
            return worker_id, list(categories[0]).index(worker_vote) #get binary value 0/1
            break
                
    raise ValueError('empty value')
    
    #remaining_responses = [x for x in item_responses if x[0] not in item_votes.keys()]
    #print(remaining_responses)
    
categories = pd.read_csv('data/TEMP/categories.txt', delimiter='\t', header=None)
responses = pd.read_csv('data/TEMP/workerResponses.txt', delimiter='\t', header=None)

i = "42_0"
item_votes = {}
for _ in range(10):
    worker_id, vote = get_vote(responses, item_votes, i, categories)
    item_votes[worker_id] = vote
    
print(item_votes)

{'A2HTGQE4AACVRV': 0, 'AYHHOK9GDSWNH': 0, 'A1QRQZWBL1SVEX': 0, 'A3G0MGLBT484I1': 0, 'A7NC1H5ZK7TO0': 0, 'A3MPXWFMR0Y2LK': 0, 'A11GX90QFWDLMM': 1, 'A3PIXX8DQGYHGI': 1, 'A2KONK3TIL5KVX': 0, 'A18941IO2ZZWW6': 1}


In [177]:
votes = np.random.binomial(1, .75, 3000)
matrix = np.fromfunction(lambda i, j: votes[i*j], (1000, 3), dtype=int)

votes_dict = {}
i=0
for vts in matrix:
    votes_dict[i] = {}
    k = 0
    for vt in vts:
        votes_dict[i][k] = [vt]
        k += 1
    
    i += 1
    
accs, ps = expectation_maximization(3, 1000, alg_utils.input_adapter(votes_dict))
np.mean(accs)

0.8338976954244961

In [403]:
datasets_accs = []

In [404]:
datasets = [
    #{'folder':'amazon-sentiment-dataset', 'filename':'1k_amazon_reviews_crowdsourced_lemmatized_min3votes.csv','predicates': ['is_negative', 'is_book']},
    #{'folder':'loneliness-dataset-2018', 'filename':'loneliness-dataset-2018.csv','predicates': ['oa_predicate', 'study_predicate']},
    #{'folder':'ohsumed_data', 'filename':'ohsumed_C14_C23_1grams.csv','predicates': ['C14', 'C23']},
    {'folder':'BarzanMozafari', 'filename':'ground_truth_normalized.csv','predicates':['gt']},
    {'folder':'RTE', 'filename':'ground_truth_normalized.csv','predicates':['gt']},
    {'folder':'SpamCF', 'filename':'ground_truth_normalized.csv','predicates':['gt']},
    {'folder':'TEMP', 'filename':'ground_truth_normalized.csv','predicates':['gt']}, 
    {'folder':'WVSCM', 'filename':'ground_truth_normalized.csv','predicates':['gt']},
]

for dataset_key in range(len(datasets)):
    dataset = datasets[dataset_key]
    
    folder = dataset['folder']
    path = f'data/{folder}/workerResponses.txt'
    votes = pd.read_csv(path, delimiter='\t', header=None)
    path = f'data/{folder}/groundTruth.txt'
    ground_truth = pd.read_csv(path, delimiter='\t', header=None)
    path = f'data/{folder}/categories.txt'
    categories = pd.read_csv(path, delimiter='\t', header=None)
    
    workers_accuracy = []
    for worker_id in np.unique(votes[0]):
        
        worker_votes = votes[votes[0] == worker_id]
        worker_successes = 0
        worker_tasks = 0
               
        for index, worker_vote_info in worker_votes.iterrows():
            
            worker_vote_task = worker_vote_info[1]
            worker_vote = worker_vote_info[2]
            
            gt_val = ground_truth[ground_truth[0]==worker_vote_task][1]
            
            if len(gt_val) > 0:
                worker_tasks += 1
                if worker_vote == gt_val.item():
                    worker_successes += 1
        #endfor
        
        if worker_tasks > 0:
            workers_accuracy.append(worker_successes / worker_tasks)
            
      
    classes_normalized = [x for x in classes[0]]
    gt_normalized = [classes_normalized.index(x) for x in gt[1]]
    
    datasets_accs.append((folder, gt_normalized, workers_accuracy))
        
datasets_accs

[('BarzanMozafari',
  [1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   1,
  

In [15]:
gt, accs = alg_utils.get_real_dataset_data("WVSCM")
sum(gt)/len(gt)

0.3625

0.5

In [292]:
import numpy as np

#s        - number of actions
#n        - number of states
#r        - number of observations
#A[a,i,j] - probability of transition from state i to state j given action a
#O[a,j,o] - probability of observing o when entering state j given action a
#pi[j]    - probability that the initial state is j
#Act      - a list of actions
#Obs      - a list of observations, the j-th observation corresponds to the 
#           j-th action

class POMDP:
    def __init__(self, Obs, Act, A, O, pi):
        self.T = len(Obs)
        self.Obs = Obs
        self.Act = Act
        self.A = A
        self.O = O
        self.pi = pi
        self.s = self.A.shape[0]
        self.n = self.A.shape[1]
        self.r = self.O.shape[2]
        self.alpha = np.zeros((self.T+1,self.n))
        self.beta = np.zeros((self.T+1,self.n))
        self.chi = np.zeros((self.T+1,self.n))
        self.xi = np.zeros((self.T+1,self.n,self.n))

    def alpha_beta_calc(self):
        for j in range(self.n):
            self.alpha[0,j] = self.pi[j]
            self.beta[self.T,j]= 1.0            

        for t in range(1, self.T+1):
            for j in range(self.n):
                self.alpha[t,j] = sum([self.A[self.Act[t- 
                1],i,j]*self.alpha[t-1,i]*self.O[self.Act[t-1],j,self.Obs[t- 
                1]] for i in range(self.n)])
                self.beta[self.T-t,j] = sum([self.A[self.Act[self.T- 
                t],j,i]*self.O[self.Act[self.T-t],i,self.Obs[self.T- 
                t]]*self.beta[self.T- t+1,i] for i in range(self.n)])

    def xi_chi_calc(self):      
        for t in range(0, self.T+1):
            l = sum([self.alpha[t,k]*self.beta[t,k] for k in range(self.n)])

            for i in range(self.n):
                self.chi[t,i]=self.alpha[t,i]*self.beta[t,i]/l
                for j in range(self.n):
                    self.xi[t,i,j]=(self.alpha[t-1,i]*self.A[self.Act[t- 
                    1],i,j]*self.O[self.Act[t-1],j,self.Obs[t- 
                    1]]*self.beta[t,j])/l

    def pi_calc(self):
        for j in range(self.n):
            self.pi[j] = self.chi[0,j]

    def A_calc(self):        
        for i in range(self.n):
            for j in range(self.n):
                for a in range(self.s):
                    nom = sum([self.xi[t,i,j] for t in range(1,self.T+1) if 
self.Act[t-1] == a])
                    denom = sum([self.chi[t-1,i] for t in range(1,self.T+1) 
if self.Act[t-1] == a])
                    self.A[a,i,j] = nom/denom

    def O_calc(self):
        for j in range(self.n):
            for k in range(self.r):
                for a in range(self.s):
                    nom = sum([self.chi[t,j] for t in range(1, self.T+1) if 
(self.Act[t-1] == a) and (self.Obs[t-1] == k)])
                    denom = sum([self.chi[t,j] for t in range(1, self.T+1) 
if self.Act[t-1] == a])
                    self.O[a,j,k] = nom/denom
def sim(l, A, O, pi):
    obs=[]
    act=[]
    if np.random.rand()>pi[0]: curr = 1
    else: curr = 0
    states = [curr]
    for _ in range(l):
        a = np.random.randint(0,2)

        if np.random.rand()<A[a, curr, 1]: curr = 1
        else: curr = 0
        if np.random.rand()<O[a, curr, 1]: o = 1
        else: o =0
        obs.append(o)
        act.append(a)
        states.append(curr)
    return act,obs,states

def generate(length = 1000):
    A = np.random.rand(2,2,2)
    O = np.random.rand(2,2,2)
    for i in range(2):
        for j in range(2):
            A[i,j,1]=1.0-A[i,j,0]
            O[i,j,1]=1.0-O[i,j,0]

    pi = np.random.rand(2)    
    pi[1]=1.0-pi[0]
    act,obs,states = sim(length, A, O, pi)
    return A, O, pi, act,obs,states

#generate random pomdp and a simulation of a given length
A, O, pi,act,obs,states = generate(100)

#initialize matrices randomly
A, O, pi,act1,obs1,states1 = generate()
pomdp = POMDP(obs, act, A, O, pi)

for _ in range(100):
    pomdp.alpha_beta_calc()        
    pomdp.xi_chi_calc()
    pomdp.A_calc()
    pomdp.O_calc()
    pomdp.pi_calc()
    
    


In [370]:
import random
import math
min_means = []
max_means = []
means = []
dot = .3
for _ in range(1000):
    min_means.append(min([.5 * (1 + math.pow(0.5, x)) for x in np.random.gamma(4,dot, 1000)]))
    max_means.append(max([.5 * (1 + math.pow(0.5, x)) for x in np.random.gamma(4,dot, 1000)]))
    means.append(np.mean([.5 * (1 + math.pow(0.5, x)) for x in np.random.gamma(4,dot, 1000)]))
    
print(np.mean(min_means))
print(np.mean(max_means))
print(np.mean(means))

0.5302678163430105
0.961456357245375
0.7349024096495098


In [484]:
a = ['1']
str(1) in a

True

In [505]:
class Test():
    
    def __init__(self, tst):
        self.worker_id = 1123123
        self.judgment = 1
        print('2')
        
    def __init__(self):
        print("tst")

tst = ['4', '3']
tst.index('3')

1

In [568]:
import random
d = 0.5
accs = [(1.0 / 2) * (1.0 + (1.0 - d) ** gamma) for gamma in np.random.gamma(4., .5, 1000)]

np.mean(accs)

0.654025080234964

In [21]:
import yaml
import yamlordereddictloader
 
with open("/Users/pmaglione/Repos/crowd-datasets/WaterBird2/classes.yaml") as f:
    yaml_data = yaml.load(f, Loader=yamlordereddictloader.Loader)

#print(yaml_data)
 
for item_id, votes in yaml_data.items():
    print(f"{item_id} {int(votes)}")
    
#for item_id, votes in yaml_data.items():
#    for worker_id, vote in votes.items():
#        print(f"{item_id}\t{worker_id}\t{int(vote)}")



0 1
1 0
2 3
3 2
4 2
5 0
6 0
7 1
8 3
9 4
10 0
11 2
12 3
13 0
14 2
15 4
16 1
17 2
18 4
19 1
20 1
21 1
22 3
23 3
24 3
25 4
26 3
27 2
28 0
29 0
30 3
31 2
32 2
33 0
34 4
35 4
36 3
37 0
38 1
39 1
40 0
41 0
42 2
43 1
44 3
45 4
46 3
47 0
48 1
49 3
50 4
51 1
52 0
53 2
54 0
55 1
56 0
57 2
58 3
59 0
60 4
61 1
62 2
63 0
64 0
65 0
66 2
67 2
68 1
69 1
70 1
71 2
72 1
73 1
74 4
75 3
76 4
77 4
78 1
79 2
80 1
81 1
82 4
83 3
84 4
85 1
86 4
87 1
88 2
89 4
90 2
91 3
92 2
93 3
94 3
95 1
96 3
97 3
98 2
99 1
100 0
101 3
102 3
103 2
104 4
105 2
106 3
107 0
108 2
109 4
110 4
111 1
112 3
113 2
114 1
115 0
116 4
117 4
118 0
119 0
120 2
121 4
122 1
123 0
124 4
125 4
126 0
127 1
128 1
129 4
130 4
131 4
132 2
133 3
134 4
135 2
136 0
137 4
138 3
139 1
140 2
141 0
142 0
143 3
144 3
145 3
146 4
147 4
148 4
149 0
150 1
151 2
152 1
153 3
154 0
155 1
156 3
157 0
158 0
159 4
160 2
161 0
162 1
163 1
164 0
165 3
166 4
167 4
168 0
169 1
170 3
171 2
172 0
173 4
174 2
175 2
176 0
177 0
178 1
179 4
180 0
181 1
182 0
183 2
184 4
